# 필요한 라이브러리 import

In [1]:
import csv
import pandas as pd
import os
import re
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from collections import defaultdict
import warnings
import pickle
plt.rc('font',family='Malgun Gothic')
plt.rc('font',size=10)
from sklearn.metrics import mean_absolute_error,mean_squared_error
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

from tensorflow.keras.models import load_model
import tensorflow as tf
import joblib
warnings.filterwarnings(action='ignore')

In [2]:
with open('./models/midfielder/midfielder_po.pickle','rb') as f:
    midfielder_po = pickle.load(f)

midfielder_model = load_model('./models/midfielder/midfielder_nn.h5')

premier_df = pd.read_csv('./datas/NEW_FM_DATA_MERGED/2021-2022/2021-2022-PremierLeague.csv')
premier_df.head(1)

,이름,시즌,포지션,구단,리그,골/90,도움/90,임대 기간,승,무,...,공중 A/90,태클 성공률,경기당 패스 시도/90,경기당 패스 성공/90,K Ps/90,태클 성공/90,드리블 성공/90,헤더 성공/90,Cr A/90,평균획득승점/경기
0,Teemu Pukki,2021-2022,공격수,노리치,잉글랜드 프리미어 리그,0.312655,0.08933,0,7,11,...,7.28,100,30.64,26.0,0.8,0.88,0.98,3.39,0.58,0.94


In [3]:
premier_df.rename(columns={'Cr C/A':'크로스 성공률','FA':'파울 획득','Int/90':'인터셉트/90','공중 A/90':'공중볼시도/90','K Ps/90':'키패스/90','Cr A/90':'크로스 시도/90'},inplace=True)
premier_df = premier_df.astype({'파울 획득':np.int64,'태클 성공률':np.int64})
premier_df.head(1)

,이름,시즌,포지션,구단,리그,골/90,도움/90,임대 기간,승,무,...,공중볼시도/90,태클 성공률,경기당 패스 시도/90,경기당 패스 성공/90,키패스/90,태클 성공/90,드리블 성공/90,헤더 성공/90,크로스 시도/90,평균획득승점/경기
0,Teemu Pukki,2021-2022,공격수,노리치,잉글랜드 프리미어 리그,0.312655,0.08933,0,7,11,...,7.28,100,30.64,26.0,0.8,0.88,0.98,3.39,0.58,0.94


In [4]:
midfielder_df = premier_df[premier_df['포지션']=='미드필더']
midfielder_df = midfielder_df.reset_index(drop=True)

In [5]:
# Goal_Shoot_linear
midfielder_df['GSL'] = (midfielder_df['경기당 유효 슈팅/90']+0.5) + (midfielder_df['경기당 슈팅/90']) + (midfielder_df['골/90']*1.3)

In [6]:
# PSTL = Pass sucess try linear

midfielder_df['PSTL'] = (midfielder_df['경기당 패스 성공/90']) + (midfielder_df['경기당 패스 시도/90']*1.15)

In [7]:
independent_value ={'GSL','PSTL','공중볼시도/90','기회 창출/90','도움/90','드리블 성공/90','반칙 수/90','키패스/90','태클 성공/90'}

dependent_value = {'평균획득승점/경기'}

In [8]:
midfielder_df.loc[:,independent_value] = midfielder_po.transform(midfielder_df.loc[:,independent_value])

In [9]:
midfielder_df = midfielder_df.reset_index(drop=True)

In [10]:
midfielder_df['예측PPG']=midfielder_model.predict(midfielder_df.loc[:,independent_value])

In [11]:
midfielder_df.sort_values(by='예측PPG',ascending=False).head(5)

,이름,시즌,포지션,구단,리그,골/90,도움/90,임대 기간,승,무,...,경기당 패스 성공/90,키패스/90,태클 성공/90,드리블 성공/90,헤더 성공/90,크로스 시도/90,평균획득승점/경기,GSL,PSTL,예측PPG
3,Mathias Normann,2021-2022,미드필더,노리치,잉글랜드 프리미어 리그,0.029297,-1.954613,21/7/5 - 22/6/30,7,13,...,54.81,4.526316,0.626198,-0.194427,2.48,0.41,0.92,-1.129696,81.033279,20.392891
35,Kalvin Phillips,2021-2022,미드필더,리즈,잉글랜드 프리미어 리그,0.026714,-1.920409,0,12,12,...,53.08,4.052632,0.428115,0.608792,2.57,0.69,1.12,-0.452417,79.338513,20.036650
66,Frank Onyeka,2021-2022,미드필더,브렌트포드,잉글랜드 프리미어 리그,0.028116,-1.958361,0,13,3,...,50.55,1.631579,0.300319,7.677124,2.81,0.48,1.17,-1.014307,74.936218,19.445843
60,Yves Bissouma,2021-2022,미드필더,브라이튼,잉글랜드 프리미어 리그,0.233463,-1.908653,0,10,8,...,48.71,2.526316,0.389776,6.552616,3.04,0.55,1.06,0.631385,74.148439,19.218233
62,Steven Alzate,2021-2022,미드필더,브라이튼,잉글랜드 프리미어 리그,0.041303,-2.047619,0,9,8,...,49.44,5.157895,0.498403,5.347787,1.74,0.45,1.03,-0.496070,73.966773,19.128456


In [12]:
midfielder_min = np.min(midfielder_df['예측PPG'].tolist())
midfielder_min

10.5370454788208

In [13]:
midfielder_df['WAR'] = midfielder_df['예측PPG'].map(lambda x : x-midfielder_min+1)

In [14]:
midfielder_df.loc[:,independent_value] = midfielder_po.inverse_transform(midfielder_df.loc[:,independent_value])

In [15]:
midfielder_war = midfielder_df.sort_values(by='WAR',ascending=False).copy()
midfielder_war = midfielder_war.reset_index(drop=True)

In [16]:
midfielder_war[['이름','키패스/90','기회 창출/90', '도움/90','WAR']].sort_values(by='WAR',ascending=False).head(10)

,이름,키패스/90,기회 창출/90,도움/90,WAR
0,Mathias Normann,1.08,0.18,0.058594,10.855845
1,Kalvin Phillips,0.99,0.19,0.080142,10.499604
2,Frank Onyeka,0.53,0.42,0.056232,9.908797
3,Yves Bissouma,0.70,0.20,0.087549,9.681188
4,Steven Alzate,1.20,0.33,0.000000,9.591411
5,John McGinn,0.98,0.41,0.182788,9.582253
6,Harry Winks,1.21,0.15,0.050364,9.517623
7,Boubakary Soumaré,1.03,0.29,0.114869,9.385770
8,Tanguy Ndombele,1.11,0.46,0.191652,9.341191
9,Joe Willock,0.97,0.29,0.171429,9.307996
